#### Необходимо проверить два предположения:

- Зависит ли вероятность оплаты от выбранного пользователем уровня сложности?
- Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности?

Проверку производите на основе данных пользователей, которые зарегистрировались в 2018 году (с 1 января по 31 декабря 2018 года включительно).

#### План действий:

1. Получите данные из файла.
2. Произведите обзор данных и преобразование данных, если оно необходимо.
3. Выделите группы пользователей по уровню сложности.
4. Рассчитайте для каждой группы процент оплат.
5. Для каждой группы сделайте по два датафрейма: один — с событиями выбора уровня сложности, другой — с событиями оплаты.
6. Объедините датафреймы в рамках одной группы и найдите разницу во времени между событиями регистрации и оплаты.
7. Рассчитайте среднее время между событиями.

In [1]:
import pandas as pd

In [2]:
from datetime import datetime, timedelta

#### 1. Получаем данные

In [3]:
events = pd.read_csv('events.csv')
purchases = pd.read_csv('purchase.xls')

#### 2. Обзор данных

In [4]:
display(events)

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585
...,...,...,...,...,...,...
252329,281232,level_choice,hard,2020-07-02T10:02:15,NaN,87439
252330,281233,level_choice,medium,2020-07-02T11:38:52,NaN,87488
252331,281234,pack_choice,NaN,2020-07-02T11:42:14,NaN,87488
252332,281235,tutorial_start,NaN,2020-07-02T13:32:58,86127.0,87464


In [5]:
display(purchases)

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100
...,...,...,...,...
5951,21625,87331,2020-07-06T09:02:07,50
5952,21626,87418,2020-07-06T14:16:37,100
5953,21627,87431,2020-07-06T22:48:59,50
5954,21628,87363,2020-07-07T05:38:56,100


Пользователи, зарегистрировавшиеся в 2018 году, в датафрейме events.

In [6]:
condition1 = (events.start_time>='2018-01-01') & (events.start_time<'2019-01-01') & (events.event_type=='registration')
registered = events[condition1].user_id.to_list() # список пользователей, зарег. в 2018
events18 = events[events.user_id.isin(registered)]
display(events18)

,id,event_type,selected_level,start_time,tutorial_id,user_id
51405,80308,registration,NaN,2018-01-01T03:48:40,NaN,27832
51406,80309,registration,NaN,2018-01-01T04:07:25,NaN,27833
51407,80310,registration,NaN,2018-01-01T08:35:10,NaN,27834
51408,80311,registration,NaN,2018-01-01T11:54:47,NaN,27835
51409,80312,registration,NaN,2018-01-01T13:28:07,NaN,27836
...,...,...,...,...,...,...
118359,147262,tutorial_start,NaN,2019-01-01T03:26:18,49554.0,47756
118360,147263,tutorial_finish,NaN,2019-01-01T03:29:25,49554.0,47756
118362,147265,level_choice,easy,2019-01-01T05:04:52,NaN,47755
118363,147266,level_choice,medium,2019-01-01T05:42:11,NaN,47756


In [7]:
events18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              66959 non-null  int64  
 1   event_type      66959 non-null  object 
 2   selected_level  8342 non-null   object 
 3   start_time      66959 non-null  object 
 4   tutorial_id     32954 non-null  float64
 5   user_id         66959 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 3.6+ MB


Покупки, совершенные пользователями в 2018 году, из датафрейма purchases.

In [8]:
condition2 = (events.start_time>='2018-01-01') & (events.start_time<'2019-01-01') & (events.event_type=='registration')
registered = events[condition2].user_id.to_list()
purch18 = purchases[purchases.user_id.isin(registered)]
purch18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1600 non-null   int64 
 1   user_id         1600 non-null   int64 
 2   event_datetime  1600 non-null   object
 3   amount          1600 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 62.5+ KB


In [9]:
purch18.describe()

,id,user_id,amount
count,1600.000000,1600.000000,1600.000000
mean,17645.505625,37752.766250,110.734375
std,462.038637,5822.621784,54.696628
min,16845.000000,27845.000000,25.000000
25%,17245.750000,32815.750000,50.000000
50%,17645.500000,37633.500000,100.000000
75%,18045.250000,43023.000000,150.000000
max,18452.000000,47742.000000,300.000000


#### Объединим датафреймы events и purchases для удобного анализа. Данные за 2018 год.

In [10]:
cond = (events.start_time>='2018-01-01') & (events.start_time<'2019-01-01') & (events.event_type=='registration')
registered = events[cond]['user_id'].to_list()
ev18 = events[events.user_id.isin(registered)]

purchases['event_type'] = 'purchase'
purch18 = purchases[purchases.user_id.isin(registered)]

events18 = ev18.rename(columns={'id':'event_id', 'start_time':'event_time'})
purchase18 = purch18.rename(columns={'id':'purchase_id', 'event_datetime':'purchase_time'})
total_events18 = pd.concat([events18,purchase18],sort=False)
total_events18['event_time'] = pd.to_datetime(total_events18['event_time'])
total_events18['purchase_time'] = pd.to_datetime(total_events18['purchase_time'])
total_events18.event_time = pd.to_datetime(total_events18.event_time, format = '%Y-%m-%dT%H:%M:%S')
total_events18.purchase_time = pd.to_datetime(total_events18.purchase_time, format = '%Y-%m-%dT%H:%M:%S')

In [11]:
total_events18

,event_id,event_type,selected_level,event_time,tutorial_id,user_id,purchase_id,purchase_time,amount
51405,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
51406,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
51407,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
51408,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
51409,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
2767,NaN,purchase,NaN,NaT,NaN,47498,18441.0,2019-01-02 03:48:19,100.0
2768,NaN,purchase,NaN,NaT,NaN,47647,18442.0,2019-01-02 23:26:26,150.0
2769,NaN,purchase,NaN,NaT,NaN,47554,18443.0,2019-01-03 00:36:36,50.0
2774,NaN,purchase,NaN,NaT,NaN,47742,18448.0,2019-01-04 12:51:41,50.0


In [12]:
total_events18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 51405 to 2778
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      68559 non-null  object        
 2   selected_level  8342 non-null   object        
 3   event_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   purchase_time   1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 5.2+ MB


Количество уникальных уровней в датафрейме

In [13]:
total_events18['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

#### 3. Теперь для каждого пользователя создадим список, который будет содержать выбор уровня сложности.

Легкий:

In [14]:
easy_level = total_events18[total_events18['selected_level'] == 'easy']
easy_list = easy_level.user_id.to_list()
len(easy_list)

2448

Средний:

In [15]:
medium_level = total_events18[total_events18['selected_level'] == 'medium']
medium_list = medium_level.user_id.to_list()
len(medium_list)

4645

Сложный:

In [16]:
hard_level = total_events18[total_events18['selected_level'] == 'hard']
hard_list = hard_level.user_id.to_list()
len(hard_list)

1249

Самый популярный уровень у пользователей средний.
Посмотрим нагляднее в процентах:

In [17]:
print('Легкий уровень - {}%'.format(round(len(easy_list)/(len(easy_list)+len(medium_list)+len(hard_list))*100, 2)))
print('Средний уровень - {}%'.format(round(len(medium_list)/(len(easy_list)+len(medium_list)+len(hard_list))*100, 2)))
print('Сложный уровень - {}%'.format(round(len(hard_list)/(len(easy_list)+len(medium_list)+len(hard_list))*100, 2)))

Легкий уровень - 29.35%
Средний уровень - 55.68%
Сложный уровень - 14.97%


#### 4. Посмотрим на тех, кто купил вопросы в разрезе уровня сложности в процентах:

In [18]:
purchase_easy = purchase18[purchase18['user_id'].isin(easy_list)]
perc_of_purchase_easy = purchase_easy['user_id'].nunique()/len(easy_list)
print('Количество пользователей уровня easy, купивших вопросы:', purchase_easy['user_id'].nunique())
print('Процент таких пользователей: {}%'.format(round(perc_of_purchase_easy*100,2)))

Количество пользователей уровня easy, купивших вопросы: 189
Процент таких пользователей: 7.72%


In [19]:
purchase_medium = purchase18[purchase18['user_id'].isin(medium_list)]
perc_of_purchase_medium = purchase_medium['user_id'].nunique()/len(medium_list)
print('Количество пользователей уровня medium, купивших вопросы:', purchase_medium['user_id'].nunique())
print('Процент таких пользователей: {}%'.format(round(perc_of_purchase_medium*100,2)))

Количество пользователей уровня medium, купивших вопросы: 969
Процент таких пользователей: 20.86%


In [20]:
purchase_hard = purchase18[purchase18['user_id'].isin(hard_list)]
perc_of_purchase_hard = purchase_hard['user_id'].nunique()/len(hard_list)
print('Количество пользователей уровня hard, купивших вопросы:', purchase_hard['user_id'].nunique())
print('Процент таких пользователей: {}%'.format(round(perc_of_purchase_hard*100,2)))

Количество пользователей уровня hard, купивших вопросы: 442
Процент таких пользователей: 35.39%


Как можно заметить из такой статистики, те, кто выбрал сложный уровень, охотнее покупают новые уровни, а те, кто выбрал легкие, мало заинтересованы в продолжении платной игры.

Можно также взглянуть на процент оплативших вопросы от общего кол-ва людей, которые выбрали уровень сложности:

In [21]:
print('Процент пользователей, оплативших вопросы после уровня easy: {}%'.format(round(purchase_easy['user_id'].nunique()/(len(easy_list)+len(medium_list)+len(hard_list))*100, 2)))
print('Процент пользователей, оплативших вопросы после уровня medium: {}%'.format(round(purchase_medium['user_id'].nunique()/(len(easy_list)+len(medium_list)+len(hard_list))*100, 2)))
print('Процент пользователей, оплативших вопросы после уровня hard: {}%'.format(round(purchase_hard['user_id'].nunique()/(len(easy_list)+len(medium_list)+len(hard_list))*100, 2)))

Процент пользователей, оплативших вопросы после уровня easy: 2.27%
Процент пользователей, оплативших вопросы после уровня medium: 11.62%
Процент пользователей, оплативших вопросы после уровня hard: 5.3%


Здесь ситуация другая - платят охотнее пользователи, которые выбрали средний уровень. Так получилось скорее всего потому, что таких пользователей больше половины. Статистика в разрезе только по уровням лучше отражает ситуацию.

Теперь проверим, какова разница во времени между регистрацией и оплатой для каждого уровня сложности.

#### 5. Выделим два отдельных датафрейма, один будет содержать только события registration со временем, второй - факт оплаты со временем. 

In [22]:
registration = total_events18[total_events18['event_type'] == 'registration'][['user_id', 'event_time']].rename(columns={'event_time': 'registration_time'})
registration

,user_id,registration_time
51405,27832,2018-01-01 03:48:40
51406,27833,2018-01-01 04:07:25
51407,27834,2018-01-01 08:35:10
51408,27835,2018-01-01 11:54:47
51409,27836,2018-01-01 13:28:07
...,...,...
118346,47753,2018-12-31 18:58:55
118347,47754,2018-12-31 19:14:08
118352,47755,2018-12-31 21:15:14
118355,47756,2018-12-31 23:17:30


Теперь следующий датафрейм, содержащий время оплаты.

In [23]:
purchase_with_time = total_events18[['user_id', 'purchase_time']]
purchase_with_time

,user_id,purchase_time
51405,27832,NaT
51406,27833,NaT
51407,27834,NaT
51408,27835,NaT
51409,27836,NaT
...,...,...
2767,47498,2019-01-02 03:48:19
2768,47647,2019-01-02 23:26:26
2769,47554,2019-01-03 00:36:36
2774,47742,2019-01-04 12:51:41


#### 6. Объединим эти датафреймы и добавим разницу во времени от регистрации до покупки.

In [24]:
reg_to_purchase = purchase_with_time.merge(registration, on='user_id', how='inner')
reg_to_purchase['timedelta'] = reg_to_purchase['purchase_time'] - reg_to_purchase['registration_time']
reg_to_purchase

,user_id,purchase_time,registration_time,timedelta
0,27832,NaT,2018-01-01 03:48:40,NaT
1,27833,NaT,2018-01-01 04:07:25,NaT
2,27833,NaT,2018-01-01 04:07:25,NaT
3,27833,NaT,2018-01-01 04:07:25,NaT
4,27834,NaT,2018-01-01 08:35:10,NaT
...,...,...,...,...
68554,47756,NaT,2018-12-31 23:17:30,NaT
68555,47756,NaT,2018-12-31 23:17:30,NaT
68556,47756,NaT,2018-12-31 23:17:30,NaT
68557,47756,NaT,2018-12-31 23:17:30,NaT


In [25]:
reg_to_purchase['timedelta'].describe()

count                         1600
mean        4 days 01:01:56.595000
std      2 days 04:49:55.587574048
min                0 days 04:36:58
25%                2 days 04:27:28
50%         3 days 19:48:53.500000
75%         5 days 16:48:57.250000
max               11 days 00:35:04
Name: timedelta, dtype: object

Создадим датафрейм с выбором уровня и объединим его с датафреймом time_to_purchase.

In [27]:
level_choice = total_events18[total_events18['event_type'] == 'level_choice'][['user_id', 'selected_level']]
time_with_levels = reg_to_purchase[['user_id', 'timedelta']].merge(level_choice, on='user_id', how='inner')
time_with_levels

,user_id,timedelta,selected_level
0,27835,NaT,medium
1,27835,NaT,medium
2,27835,NaT,medium
3,27835,NaT,medium
4,27835,NaT,medium
...,...,...,...
47551,47756,NaT,medium
47552,47756,NaT,medium
47553,47756,NaT,medium
47554,47756,NaT,medium


#### 7. Теперь будем анализировать разницу во времени по каждому уровню сложности.

Легкий:

In [28]:
easy_delta = time_with_levels[time_with_levels['selected_level'] == 'easy']
easy_delta['timedelta'].mean()

Timedelta('3 days 22:10:23.211640211')

Средний:

In [29]:
medium_delta = time_with_levels[time_with_levels['selected_level'] == 'medium']
medium_delta['timedelta'].mean()

Timedelta('4 days 06:12:06.576883384')

Сложный:

In [30]:
hard_delta = time_with_levels[time_with_levels['selected_level'] == 'hard']
hard_delta['timedelta'].mean()

Timedelta('3 days 14:55:19.257918552')

#### Итак, получившийся анализ разницы во времени между регистрацией и покупкой нам показал, что среднее время для легкого уровня и для сложного совсем небольшой - с небольши отрывом лидирует группа "сложных". Предполагая, что прохождение сложных уровней может занять больше времени в принципе, чем прохождение легких, то этот результат можно считать более эффективным. Средний уровень решается на покупку позже почти на сутки.

Попробуем сравнить время от выбора уровня до оплаты, чтобы понять лучше поведение группы.

In [31]:
level_time = total_events18[total_events18['event_type'] == 'level_choice'][['user_id', 'event_time', 'selected_level']].rename(columns={'event_time': 'level_choice_time'})
lvl_to_purchase = purchase_with_time.merge(level_time, on='user_id', how='inner')
lvl_to_purchase['timedelta'] = lvl_to_purchase['purchase_time'] - lvl_to_purchase['level_choice_time']
lvl_to_purchase

,user_id,purchase_time,level_choice_time,selected_level,timedelta
0,27835,NaT,2018-01-01 20:37:22,medium,NaT
1,27835,NaT,2018-01-01 20:37:22,medium,NaT
2,27835,NaT,2018-01-01 20:37:22,medium,NaT
3,27835,NaT,2018-01-01 20:37:22,medium,NaT
4,27835,NaT,2018-01-01 20:37:22,medium,NaT
...,...,...,...,...,...
47551,47756,NaT,2019-01-01 05:42:11,medium,NaT
47552,47756,NaT,2019-01-01 05:42:11,medium,NaT
47553,47756,NaT,2019-01-01 05:42:11,medium,NaT
47554,47756,NaT,2019-01-01 05:42:11,medium,NaT


In [32]:
easy_choice_delta = lvl_to_purchase[lvl_to_purchase['selected_level'] == 'easy']
medium_choice_delta = lvl_to_purchase[lvl_to_purchase['selected_level'] == 'medium']
hard_choice_delta = lvl_to_purchase[lvl_to_purchase['selected_level'] == 'hard']
print('easy: ',easy_choice_delta['timedelta'].mean())
print('medium: ',medium_choice_delta['timedelta'].mean())
print('hard: ', hard_choice_delta['timedelta'].mean())

easy:  3 days 14:58:52.941798941
medium:  3 days 23:14:13.165118679
hard:  3 days 07:20:41.420814479


#### В этом разрезе разница между выбором уровня и оплатой небольшая, и можно сделать вывод, что выбор уровня на скорость, наверное не влияет.

### С учетом процента оплат внутри каждой группы и средним временем от регистрации до покупки, эффективнее всего получается продавать пакеты вопросов пользователям, выбравших сложный уровень.

### Средний уровень выбирает большинство пользователей, но покупают с меньшей охотой, чем "сложные", а также дольше принимают это решение. Возможно стоит таких пользователей больше мотивировать переходить на сложный уровень.

### Те, кто вбрал легкий уровень, покупают дальше очень мало и для этой группы тоже довольно долго принимают решение. Возможно, проходя легкие уровни, пользователи быстро теряют интерес. Тут, возможно, стоило бы либо усложнить уровни, либо вообще в игре оставить только medium и hard.